In [1]:
%%init_spark
# Configure Spark to use a local master
launcher.master = "local"

In [2]:
import org.apache.spark.sql.expressions.Window

Intitializing Scala interpreter ...

Spark Web UI available at http://9a9c2e1d7612:4041
SparkContext available as 'sc' (version = 3.0.0, master = local, app id = local-1597114052336)
SparkSession available as 'spark'


import org.apache.spark.sql.expressions.Window


In [3]:
import org.apache.spark.sql.functions.{window, column, desc, col,count}

import org.apache.spark.sql.functions.{window, column, desc, col, count}


In [4]:
val df = spark.read.format("csv").option("header", "true").load("data/demo.csv")

df: org.apache.spark.sql.DataFrame = [airline_id: string, name: string ... 6 more fields]


In [5]:
df.printSchema()

root
 |-- airline_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- alias: string (nullable = true)
 |-- iata: string (nullable = true)
 |-- icao: string (nullable = true)
 |-- callsign: string (nullable = true)
 |-- country: string (nullable = true)
 |-- active: string (nullable = true)



### 1. Generar un diccionario a partir de la columna country (country_id,country)

### Se crea la tabla country

In [8]:
val table_country = df.select(upper(col("country")))
                       .filter("country is not NULL").distinct().orderBy("country")
                       .withColumnRenamed("upper(country)", "country")

table_country: org.apache.spark.sql.DataFrame = [country: string]


### Se agrega un identificador a la tabla

In [10]:
val window = Window.orderBy(col("country"))

window: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@31609366


In [16]:
val table_country_id = table_country.withColumn("country_id",row_number().over(window))

table_country_id: org.apache.spark.sql.DataFrame = [country: string, country_id: int]


In [17]:
table_country_id.printSchema()

root
 |-- country: string (nullable = true)
 |-- country_id: integer (nullable = true)



### 2. Escribir el diccionario como CSV desde spark

### Se crea el archivo country.csv

In [20]:
table_country_id.write
        .format("com.databricks.spark.csv")
        .option("header", "true")
        .mode("overwrite")
        .save("data/country")


### Se lee el archivo country.csv

In [21]:
val df_country = spark.read.format("csv").option("header", "true").load("data/country/country.csv")

df_country: org.apache.spark.sql.DataFrame = [country: string, country_id: string]


### 3. Leer el CSV generado y hacer inner join con demo.csv

### Se crea JOIN

In [27]:
val df_asDemo    = df.as("dfdemo")
val df_asCountry = df_country.as("dfcountry")

df_asDemo: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [airline_id: string, name: string ... 6 more fields]
df_asCountry: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [country: string, country_id: string]


In [28]:
val df_join = df_asDemo.join(df_asCountry,col("dfdemo.country") === col("dfcountry.country"),"left_outer")

df_join: org.apache.spark.sql.DataFrame = [airline_id: string, name: string ... 8 more fields]


In [29]:
df_join.printSchema()

root
 |-- airline_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- alias: string (nullable = true)
 |-- iata: string (nullable = true)
 |-- icao: string (nullable = true)
 |-- callsign: string (nullable = true)
 |-- country: string (nullable = true)
 |-- active: string (nullable = true)
 |-- country: string (nullable = true)
 |-- country_id: string (nullable = true)



In [30]:
df_join.count()

res8: Long = 6162


In [32]:
df.count()

res10: Long = 6162


### Parar session de SPARK

In [33]:
spark.stop()